In [ ]:
import src.utils as ut
from src.data import load_dataset_and_make_dataloaders
from src.sigma import build_sigma_schedule
from src.common import euler_sample
from pathlib import Path
import torch
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
wd = Path("/storage/homefs/ds21n601/diffusion_project_DL/runs/20251210_155344_fm_cond")

_, info = load_dataset_and_make_dataloaders(dataset_name="FashionMNIST", root_dir="../data", batch_size=32)
chkp = ut.find_latest_checkpoint(wd / "checkpoints")
model = ut.build_model_for_sampling(wd / "config.yaml", device, info)
model = ut.load_ckpt_into_model(model, chkp, device)

In [ ]:
channels = info.image_channels
H = info.image_size
sigma_data = float(info.sigma_data)
n_images = 32

sigmas = build_sigma_schedule(100)
sigmas = sigmas.to(device)

samples = euler_sample(model, sigmas, n_images, channels, H, sigma_data, device, None)
imgs = samples.clamp(-1, 1).add(1).div(2)
grid_img = make_grid(imgs, nrow=min(8, n_images)).cpu()

plt.imshow(grid_img.permute(1, 2, 0))
_ = plt.axis("off")